# Classification Model using Parameters from Ad URLs
Resource: https://docs.google.com/spreadsheets/d/12f6zRQyWlPnVTHjkJwY2jZSjVVAkpaIa/edit#gid=1812140099

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from snowflake.connector import connect
import os


C:\Users\uremekn\Anaconda3\lib\site-packages\snowflake\connector\options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (6.0.1), please install a version that adheres to: 'pyarrow<3.1.0,>=3.0.0; extra == "pandas"'
  warn_incompatible_dep(


### Load the utm source/medium actual and ideal datasets

In [2]:
ctx = snowflake.connector.connect(
        user='<USER>',
        password='<PASS>',
        account='<ACCOUNT>',
        warehouse='<WAREHOUSE>',
        database='<DATABASE>',
        schema='<SCHEMA>')

In [3]:
sql_actual = """ select 
        distinct utm_sid
            , utm_source
            , utm_medium
            , sum(mau) as mau

        from (
          select
            distinct utm_concat
            , utm_sid
            , utm_source
            , utm_medium
            , count(client_sid) as mau

            from (
                select
                    distinct a.client_sid
                    , u.utm_sid
                    , u.utm_source
                    , u.utm_medium
                    , concat(u.utm_source,'_',u.utm_medium) as utm_concat
                    from ODIN_PRD.DW_ODIN.UTM_DIM u
                    join ODIN_PRD.RPT.HOURLY_TVS_AGG a on u.utm_sid = a.utm_sid
                    join ODIN_PRD.DW_ODIN.HOUR_DIM h on h.hour_sid = a.hour_sid

              union

                select
                    distinct a.client_sid
                    , u.utm_sid
                    , u.utm_source
                    , u.utm_medium
                    , concat(u.utm_source,'_',u.utm_medium) as utm_concat
                    from ODIN_PRD.DW_ODIN.S_UTM_DIM u
                    join ODIN_PRD.RPT.S_HOURLY_INACTIVE_TVS_AGG a on u.utm_sid = a.utm_sid
                    join ODIN_PRD.DW_ODIN.HOUR_DIM h on h.hour_sid = a.hour_sid
                    )

            group by 1,2,3,4
            )

        group by 1,2,3; """

actual = pd.read_sql_query(sql_actual, con=ctx)

In [4]:
# pull data for source and medium ideal values
sql_ideal = """ 
    SELECT 
        SOURCE_IDEAL
        , MEDIUM_IDEAL
        , CHANNEL
        , ACQUISITION
        , BRAND
        , CONTENT
        , CRM
        , DISTRIBUTION
        , OTHER
        , UNDEFINED
        , ORGANIC_PAID
        
    FROM SANDBOX.ANALYSIS_MARKETING.UTM_MAPPING; """

ideal = pd.read_sql_query(sql_ideal, con=ctx)

### Cosine Similarity Function

In [5]:
def similarity(utm_actual, utm_ideal):
    
    # filter data to utms with maus > 100
    actual_filtered = actual[actual['MAU'] > 100]
    actual_filtered[utm_actual] = actual_filtered[utm_actual].astype(str).apply(str.lower).replace({'nan':'na','none':'na'})  

    # pull actual and ideal source and medium lists to prep for cosine similarity calculation
    ideal_list = list(set(ideal[utm_ideal].dropna()))
    actual_list = list(set(actual_filtered[utm_actual].dropna()))
    
    # cosine similarity for utm source
    def word2vec(word):
        from collections import Counter
        from math import sqrt

        # count the characters in word
        cw = Counter(word)
        # precomputes a set of the different characters
        sw = set(cw)
        # precomputes the "length" of the word vector
        lw = sqrt(sum(c*c for c in cw.values()))

        # return a tuple
        return cw, sw, lw

    def cosdis(v1, v2):
        # which characters are common to the two words?
        common = v1[1].intersection(v2[1])
        # by definition of cosine distance we have
        if v1[2] > 0 and v2[2] > 0:
            return sum(v1[0][ch]*v2[0][ch] for ch in common)/v1[2]/v2[2]
#         else:
#             continue

    threshold = 0.0     # if needed

    similarity = []
    for key in actual_list:
        for word in ideal_list:
            try:
                # print(key)
                # print(word)
                res = cosdis(word2vec(word), word2vec(key))
                # print(res)
                similarity.append([word, key, res])
                #print("The cosine similarity between : {} and : {} is: {}".format(word, key, res*100))
                # if res > threshold:
                #     print("Found a word with cosine distance > 80 : {} with original word: {}".format(word, key))
            except IndexError:
                pass

    df = pd.DataFrame(similarity, columns=[utm_actual[4:] + '_IDEAL', utm_actual[4:] + '_ACTUAL', utm_actual[4:] + '_SIMILARITY_SCORE'])
    df.sort_values(utm_actual[4:] + '_SIMILARITY_SCORE', ascending=False, inplace=True)
    
    df_final = pd.merge(df, actual_filtered['UTM_SID'], how='left', left_on=df[utm_actual[4:] + '_ACTUAL'], right_on=actual_filtered[utm_actual])
    del df_final['key_0']

    df_final.sort_values(utm_actual[4:] + '_SIMILARITY_SCORE', ascending=False)
    df_final = df_final[df_final[utm_actual[4:] + '_SIMILARITY_SCORE'] >= 0.9]

    return df_final
        
source_df = similarity('UTM_SOURCE', 'SOURCE_IDEAL')
medium_df = similarity('UTM_MEDIUM', 'MEDIUM_IDEAL')

<ipython-input-5-b9a2305f4dac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_filtered[utm_actual] = actual_filtered[utm_actual].astype(str).apply(str.lower).replace({'nan':'na','none':'na'})
<ipython-input-5-b9a2305f4dac>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  actual_filtered[utm_actual] = actual_filtered[utm_actual].astype(str).apply(str.lower).replace({'nan':'na','none':'na'})


### Merge Source and Medium datasets

In [6]:
# merge source and medium datasets
source_medium = pd.merge(source_df, medium_df[['MEDIUM_IDEAL','MEDIUM_ACTUAL','MEDIUM_SIMILARITY_SCORE','UTM_SID']], how='outer', left_on=source_df['UTM_SID'], right_on=medium_df['UTM_SID'])
del source_medium['key_0']

source_medium['SOURCE_ACTUAL'] = source_medium['SOURCE_ACTUAL'].astype(str).apply(str.lower).replace('nan','na')
source_medium['MEDIUM_ACTUAL'] = source_medium['MEDIUM_ACTUAL'].astype(str).apply(str.lower).replace('nan','na')

# create mapping field to join with the ideal datasets for pulling in categories like channel and paid/organic
source_medium['MAPPING'] = source_medium['SOURCE_IDEAL'].astype(str) + '_' + source_medium['MEDIUM_IDEAL'].astype(str)
ideal['MAPPING'] = ideal['SOURCE_IDEAL'].astype(str) + '_' + ideal['MEDIUM_IDEAL'].astype(str)

source_medium_categories = pd.merge(source_medium, ideal[['CHANNEL','ACQUISITION','BRAND','CONTENT','CRM','DISTRIBUTION','OTHER','UNDEFINED','ORGANIC_PAID']], how='left', left_on=source_medium['MAPPING'], right_on=ideal['MAPPING'])
del source_medium_categories['key_0']

final = source_medium_categories[source_medium_categories['UNDEFINED'].notnull()]
final['UTM_SID'] = final[['UTM_SID_x', 'UTM_SID_y']].bfill(axis=1).iloc[:, 0]
del final['UTM_SID_x']
del final['UTM_SID_y']
del final['MAPPING']

final['UTM_SID'] = final['UTM_SID'].astype(int)

<ipython-input-6-69da4ebfd24d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['UTM_SID'] = final[['UTM_SID_x', 'UTM_SID_y']].bfill(axis=1).iloc[:, 0]
<ipython-input-6-69da4ebfd24d>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final['UTM_SID'] = final['UTM_SID'].astype(int)


### Upload data into Snowflake/ODIN

In [7]:
ctx.cursor().execute(
    """ CREATE TABLE IF NOT EXISTS ANALYSIS_MARKETING.UTM_SIMILARITY_MAPPING
      ("SOURCE_IDEAL" NVARCHAR(200),
        "SOURCE_ACTUAL" NVARCHAR(200),
        "SOURCE_SIMILARITY_SCORE" FLOAT,
        "MEDIUM_IDEAL" NVARCHAR(200),
        "MEDIUM_ACTUAL" NVARCHAR(200),
        "MEDIUM_SIMILARITY_SCORE" FLOAT,
        "CHANNEL" NVARCHAR(20),
        "ACQUISITION" INTEGER,
        "BRAND" INTEGER,
        "CONTENT" INTEGER,
        "CRM" INTEGER,
        "DISTRIBUTION" INTEGER,
        "OTHER" INTEGER,
        "UNDEFINED" INTEGER,
        "ORGANIC_PAID" NVARCHAR(10),
        "UTM_SID" INTEGER,
        PRIMARY KEY ("UTM_SID")); """)

#ctx.cursor().execute(""" drop table ANALYSIS_MARKETING.UTM_SIMILARITY_MAPPING; """)
ctx.cursor().execute(""" truncate ANALYSIS_MARKETING.UTM_SIMILARITY_MAPPING; """)


#Define the table name, schema, and database you want to write to
#Note: the table, schema, and database need to already exist in Snowflake

table_name = "UTM_SIMILARITY_MAPPING"
schema = 'ANALYSIS_MARKETING'
database = 'SANDBOX'

#Combine these using the function from the Snowflake connector

write_pandas(
            conn=ctx,
            df=final,
            table_name=table_name,
            database=database,
            schema=schema
        )

ctx.close()